# Welcome to Jupyter!

In [9]:
from nltk.corpus import stopwords

class sentence:
    def __init__(self, docname, stemwords, originalwords):
        self.docname = docname
        self.stemwords = stemwords
        self.wordfreq = self.sentenceswordfreq()
        self.orignalwords = originalwords
    
    def getdocname(self):
        return self.docname
    
    def getstemwords(self):
        return self.stemwords
    
    def getoriginalwords(self):
        return self.originalwords
    
    def sentenceswordfreq(self):
        wordfreq = {}
        for word in stemwords:
            if word not in wordfreq.keys():
                wordfreq[word] = 1
            else:
                wordfreq[word] = wordfreq[word] + 1
        return wordfreq

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

class preprocessing:
    def __init__(self, docname, doc):
        self. docname = docname
        self.doc = doc
        
    def removestopwords(self):
        words = word_tokenize(doc)
        stoplist = []
        stopwords = set(stopwords.words("English"))
        for word in words:
            if word not in stopwords:
                stoplist.append(word)
        return stoplist
    
    def stemming(sentences):
         
    
        

In [57]:
import nltk
import os
import math
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import defaultdict

class sentence:
    def __init__(self, docname, stemwords, originalwords):
        self.docname = docname
        self.stemwords = stemwords
        self.wordfreq = self.sentenceswordfreq()
        self.originalwords = originalwords
    
    def getdocname(self):
        return self.docname
    
    def getstemwords(self):
        return self.stemwords
    
    def getoriginalwords(self):
        return self.originalwords
    
    def sentenceswordfreq(self):
        wordfreq = {}
        for word in self.stemwords:
            # print(word)
            wordfreq.setdefault(word, 1)
            wordfreq[word] = wordfreq[word] + 1
        return wordfreq
    
# replace doc by docname 
def preprocessing(doc):
    f = open(doc,'r')
    doc1 = f.read()
    tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')
    lines = tokenizer.tokenize(doc1.strip())
    sentences = []
    porter = nltk.PorterStemmer()
    totalnumwords = 0
    for line in lines:
        originalwords = line[:]
        line = line.strip()
        words = word_tokenize(line)
        stemmedword = [porter.stem(word) for word in words]
        stemmedword = filter(lambda x: x!='.'and x!='`'and x!=','and x!='?'and x!="'" and x!='!' and x!='''"''' and x!="''" and x!="'s", stemmedword)
        l = list(stemmedword)
        #[] is False in python
        if (l):
            s = sentence(doc, l, originalwords)
            sentences.append(s)
    return sentences

#divide by tf by totalnumwords
def termfreq(sentences):
    tf = {}
    for sent in sentences:
        wordfreq = sent.sentenceswordfreq()
        for word in wordfreq:        
            if (tf.get(word,0) == 0):
                tf[word] = wordfreq[word]
            else:
                tf[word] = tf[word] + wordfreq[word]
    return tf

def inversedocfreq(sentences):
    N = len(sentences)
    idf = 0
    idfs = {}
    words = {}
    
    for sent in sentences:

        for word in sent.getstemwords():

            if sent.sentenceswordfreq().get(word, 0) != 0:
                words[word] = words.get(word, 0)+ 1

    for word in words:
        n = words[word]

        try:
            idf = math.log10(float(N)/n)
        except ZeroDivisionError:
            idf = 0

        idfs[word] = idf
            
    return idfs

#return a dictionary with keys as tf-idf values
def tf_idf(sentences):
    tf_idf_w = defaultdict(list)
    tf = termfreq(sentences)
    idf = inversedocfreq(sentences)
#     print(idf)
    for word in tf:
        tf_idfs = tf[word]*idf[word]
        
        if tf_idf_w.get(tf_idfs, None) == None:
            tf_idf_w[tf_idfs] = [word]
        else:
            tf_idf_w[tf_idfs].append(word)
    return tf_idf_w

def cosinesim(sent1, sent2):
    s1 = set(sent1.getstemwords())
    s2 = set(sent2.getstemwords())
    total = s1.union(s2)
    l1 = []
    l2 = []
    for word in total:
        if(word in s1):
            l1.append(1)
        else:
            l1.append(0)
        if(word in s2):
            l2.append(1)
        else:
            l2.append(0)
       
    num = 0
    for i in range(len(l1)):
        num += l1[i]*l2[i]
    
    l1 = [i**2 for i in l1]
    l2 = [i**2 for i in l2]
    
    denom = (sum(l1)*sum(l2))**(0.5)
    
    try:
        return float (num/ denom)
    except ZeroDivisionError:
        return float("-inf")

def bestquery(sentences, tf_idf_w, n):
    scores = tf_idf_w.keys()
    #descending order of scores
    keys = []
    for key in scores:
        keys.append(key)
    #print(keys, 1)
    sortedkeys = sorted(keys, reverse = True)
    query = []
    done = False
    for s in sortedkeys:
        words = tf_idf_w[s]
        for word in words:
            if(len(query) < n):
                query.append(word)
            else:
                done = True
                break
        if(done):
            break
            
    return sentence("query", query, query)

def bestscoringsent(sentences, query):
    maxval = float("-inf")
    bestsent = sentences[0]
    for sent in sentences:
        similarity = cosinesim(sent,query)
        if(maxval < similarity):
            maxval = similarity
            bestsent = sent
    
    sentences.remove(bestsent)
    return bestsent
             
    
def MMR(Si,Sj,l,query):
    Sim1 = cosinesim(Si,query)
    Sim2 = float("-inf")
    for sent in Sj:
        max = cosinesim(Si,sent)
        if(max > Sim2):
            Sim2 = max
    MMRscore = l*Sim1 - (1-l)*Sim2
    return MMRscore


def selectsent(sentences, bestsent, query, summarylength, l):
    summary = [bestsent]
    sumlen = len(bestsent.getstemwords())
    s = sentences[0]
    while(sumlen < summarylength):
        MMRval = {}
        for sent in sentences:
            MMRval[sent] = MMR(sent, summary, l, query)
        
        maxscore = 0
        for k,v in MMRval.items():
            if(maxscore < v):
                maxscore = v
                s = k
        summary.append(s)
        sentences.remove(s)
        sumlen += len(s.getstemwords())
    return summary

if __name__ == '__main__':
#     print('hi')

    mainpath = os.getcwd() + "/MMR_DataSet"
    files = os.listdir(mainpath)
    sentences = []
#     print(files)
    for file in files:
        if(file == '.ipynb_checkpoints'):
            continue
        sentences += preprocessing(mainpath + "/" + file)
    #print(sentences[0].__dict__)
    TF_IDF_w = tf_idf(sentences)
#    print(TF_IDF_w.items())
    query = bestquery(sentences,TF_IDF_w, 10)
#   print(query.__dict__)
    bestsent = bestscoringsent(sentences, query)
#    print(bestsent.__dict__)
     #Vary length and lambda
    summary = selectsent(sentences, bestsent, query, 100, 0.5)
    final_summary = ""
    for sent in summary:
        final_summary = final_summary + sent.getoriginalwords()
    print(final_summary)
    completename = os.path.join(os.getcwd(), "MMRSummary.txt")
    f = open(completename, 'w+')
    f.write(final_summary)
    f.close()

But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement.“Upon taking office, the Trump administration will evaluate this case and all related aspects of the Affordable Care Act.Insurers that receive the subsidies in exchange for paying    costs such as deductibles and   for eligible consumers could race to drop coverage since they would be losing money.But on spending power and standing, the Trump administration may come under pressure from advocates of presidential authority to fight the House no matter their shared views on health care, since those precedents could have broad repercussions.


In [17]:
def max(b):
    max = b[0];
    for i in range(len(b)):
        if(max<b[i]):
            max = b[i]
    b.remove(max)
    return max;
a = [1,2]
print(max(a))
print(a)

2
[1]


In [ ]:
a_dictionary = {"a": 1, "b": 2, "c": 3}

max_key = max(a_dictionary, key=a_dictionary.get)
print(max_key)

In [ ]:
from collection import defaultdict

a = defaultdict([])
b = defaultdict(list)
a[3].append(3)
print(a)
b[3].append(3)
print(b)

In [ ]:
dic = {1:1, 2:2}
keys = []
for key in dic.keys():
    keys.append(key)
    
keys = sorted(keys, reverse = True )
print(keys)
print(dic)

This repo contains an introduction to [Jupyter](https://jupyter.org) and [IPython](https://ipython.org).

Outline of some basics:

* [Notebook Basics](../examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](../examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](../examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](../examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](../examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Running a Secure Public Notebook Server](../examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter works](../examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

You can also get this tutorial and run it on your laptop:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.anaconda.com/download):

    conda install ipython jupyter

with pip:

    # first, always upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook